### FUTURES

A future, or promise, is something that represents a pending opearion and returns straight away. 

You have seen one idea of futures in the Lazy eval" paer of this course where we created a class instance instead of executing a function. Thean calling `eval` evaluated the function. But we stopped there, not constructing a structure in which we figure where and when we should execute the `eval`, and how we should return the result to the user.

These issues become important when we are running many threads or are in a callback based system. We'll see how python handles this with threads for us, and later, how we would do this for co-routines and callbacks.

We use the `concurrent.futures` module here to do this. One creates a "thread pool" with multiple threads (or processes). One can then query their state of completion, or register callbacks to be called on successful completion or error.
Examples Adapted from Fluent Python.

In [1]:
import time, uuid, functools
def get_thing_maker(secs, item):
    time.sleep(secs)
    return str(uuid.uuid4())+str(item)
get_thing = functools.partial(get_thing_maker, 1)
def get_many(lot):
    counter=0
    for t in lot:
        thing = get_thing(t)
        counter += 1
    return counter
def serial_main(it):
    t0 = time.time()
    count = get_many(it)
    elapsed = time.time() - t0
    msg = '\n{} things got in {:.2f}s' 
    print(msg.format(count, elapsed))

In [19]:
serial_main(range(20))


20 things got in 20.01s


In [20]:
from concurrent import futures
def get_many_threaded1(it):
    workers = 10
    with futures.ThreadPoolExecutor(max_workers=workers) as executor:
        res = executor.map(get_thing, it)
    return len(list(res))
def threaded_main1(it):
    t0 = time.time()
    count = get_many_threaded1(it)
    elapsed = time.time() - t0
    msg = '\n{} things got in {:.2f}s' 
    print(msg.format(count, elapsed))

In [21]:
threaded_main1(range(20))


20 things got in 2.01s


One might think that the concurrent IO (or sleeping) case is limited by the GIL, but in both cases, the GIL is yielded. Thus there is no waiting around.

The GIL is harmless if code is being run in the context of python library IO or code running in properly coded C extensions like numpy. The time.sleep() function also releases the GIL. Python threads are totally usable in I/O-bound applications.


### Threads

### threads vs processes

On linux
- processes created by fork() have a primary thread
- thread is the unit of execution
- process is a container, can have more threads
- can be scheduled across different cores/cpus

```c
int pid;
int status = 0;
/* fork returns pid of child to parent and 0 to child*/
if (pid = fork()) {
    /* parent code */
    pid = wait(&status);
    /*wait returns child pid and status*/
} else {
    /* child  code*/
    exit(status);
}
```

- threads in a process share same address space (share it entirely)
- thread abstraction decouples resource allocation from control
- defines a single sequential execution stream with PC, stack, register values
- process handles: address space, global variables, open files, child processes, pending alarms, signals and signal handlers, accounting info
- thread handles program counter, registers, stack, and state

### Processes with concurrent futures.

CPU based processing wont release the gil, and is thus best done in a separate process. For illustration, we show what this looks like.

In [22]:
import time
def get_many_process(it, workers=None):
    if workers:
        with futures.ProcessPoolExecutor(max_workers=workers) as executor:
            res = executor.map(get_thing, it)
    else:
        with futures.ProcessPoolExecutor() as executor:
            res = executor.map(get_thing, it)
    return len(list(res))

def process_main(it, workers=None):
    t0 = time.time()
    count = get_many_process(it, workers)
    elapsed = time.time() - t0
    msg = '\n{} things got in {:.2f}s' 
    print(msg.format(count, elapsed))

In [23]:
process_main(range(20))


20 things got in 5.04s


In [24]:
process_main(range(20), workers=10)


20 things got in 2.05s


In [1]:
def fib(n):
    return fib(n - 1) + fib(n - 2) if n > 1 else n
def cpuy():
    for i in range(35):
        val = fib(i)
        print("fib({}) is {}".format(i, val))

def cpuy2():
    for i in range(35):
        val = fib(i)
        print("cpuy2 fib({}) is {}".format(i, val))

In [12]:
from threading import Thread
def thr():
    # Second thread will print the hello message. Starting as a daemon means
    # the thread will not prevent the process from exiting.
    start = time.time()
    cpuy()
    cpuy2()
    print("serial elapsed:", time.time() - start)
    start=time.time()
    #t = Thread(target=sleepy)
    #t.start()
    t2 = Thread(target=cpuy2)
    t2.start()
    # Main thread will read and process input
    cpuy()
    print("thread elapsed:", time.time() - start)

In [13]:
thr()

fib(0) is 0
fib(1) is 1
fib(2) is 1
fib(3) is 2
fib(4) is 3
fib(5) is 5
fib(6) is 8
fib(7) is 13
fib(8) is 21
fib(9) is 34
fib(10) is 55
fib(11) is 89
fib(12) is 144
fib(13) is 233
fib(14) is 377
fib(15) is 610
fib(16) is 987
fib(17) is 1597
fib(18) is 2584
fib(19) is 4181
fib(20) is 6765
fib(21) is 10946
fib(22) is 17711
fib(23) is 28657
fib(24) is 46368
fib(25) is 75025
fib(26) is 121393
fib(27) is 196418
fib(28) is 317811
fib(29) is 514229
fib(30) is 832040
fib(31) is 1346269
fib(32) is 2178309
fib(33) is 3524578
fib(34) is 5702887
cpuy2 fib(0) is 0
cpuy2 fib(1) is 1
cpuy2 fib(2) is 1
cpuy2 fib(3) is 2
cpuy2 fib(4) is 3
cpuy2 fib(5) is 5
cpuy2 fib(6) is 8
cpuy2 fib(7) is 13
cpuy2 fib(8) is 21
cpuy2 fib(9) is 34
cpuy2 fib(10) is 55
cpuy2 fib(11) is 89
cpuy2 fib(12) is 144
cpuy2 fib(13) is 233
cpuy2 fib(14) is 377
cpuy2 fib(15) is 610
cpuy2 fib(16) is 987
cpuy2 fib(17) is 1597
cpuy2 fib(18) is 2584
cpuy2 fib(19) is 4181
cpuy2 fib(20) is 6765
cpuy2 fib(21) is 10946
cpuy2 fib(22) is 177

In [14]:
import multiprocessing, time
start = time.time()
p=multiprocessing.Process(target=cpuy2)
p.start()
cpuy()
p.join()
print("mp elapsed:", time.time() - start)

cpuy2 fib(0) is 0
cpuy2 fib(8) is 21
cpuy2 fib(1) is 1
cpuy2 fib(2) is 1
cpuy2 fib(3) is 2
cpuy2 fib(4) is 3
cpuy2 fib(5) is 5
cpuy2 fib(6) is 8
cpuy2 fib(7) is 13
cpuy2 fib(9) is 34
cpuy2 fib(12) is 144
cpuy2 fib(10) is 55
cpuy2 fib(11) is 89
cpuy2 fib(13) is 233
cpuy2 fib(15) is 610
cpuy2 fib(14) is 377
cpuy2 fib(16) is 987
cpuy2 fib(18) is 2584
cpuy2 fib(17) is 1597
cpuy2 fib(19) is 4181
cpuy2 fib(20) is 6765
cpuy2 fib(21) is 10946
cpuy2 fib(22) is 17711
cpuy2 fib(23) is 28657
cpuy2 fib(24) is 46368
fib(0) is 0
fib(1) is 1
fib(2) is 1
fib(3) is 2
fib(4) is 3
fib(5) is 5
fib(6) is 8
fib(7) is 13
fib(8) is 21
fib(9) is 34
fib(10) is 55
fib(11) is 89
fib(12) is 144
fib(13) is 233
fib(14) is 377
fib(15) is 610
fib(16) is 987
fib(17) is 1597
fib(18) is 2584
fib(19) is 4181
fib(20) is 6765
fib(21) is 10946
fib(22) is 17711
fib(23) is 28657
fib(24) is 46368
fib(25) is 75025
cpuy2 fib(25) is 75025
cpuy2 fib(26) is 121393
cpuy2 fib(27) is 196418
fib(26) is 121393
fib(27) is 196418
cpuy2 fib(

Use threads for IOy stuff and processes for computey stuff

#### A URL fetcher

Example from concurrent.futures documentation

In [18]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    #executor.submit returns a future.
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://some-made-up-domain.com/' generated an exception: <urlopen error [Errno 8] nodename nor servname provided, or not known>
'http://www.foxnews.com/' page is 70624 bytes
'http://www.cnn.com/' page is 124685 bytes
'http://www.bbc.co.uk/' page is 182063 bytes
'http://europe.wsj.com/' page is 915474 bytes


## What are sockets?


From wikipedia:
>A network socket is an endpoint of a connection in a computer network. In Internet Protocol (IP) networks, these are often called Internet sockets. It is a handle (abstract reference) that a program can pass to the networking application programming interface (API) to use the connection for receiving and sending data.

- we will stick to TCP (or STREAM) sockets and to INET (or IPV4) sockets
- a client program only has a client socket
- a server program has both client and server sockets: the server socket is the thing that sits and listens for connections, spawning a client socket to deal with them.

### Why do we care?

- we'll need it for the client communications for our database
- you will want to know how to use them to make custom data servers and clients. 

Here for example, is a no-copy numpy array sharing over tcp sockets. Might be useful one day...

### A simple server and its client

In [2]:
%%file server0.py
from socket import *
s = socket(AF_INET, SOCK_STREAM)
s.bind(('', 25000))
s.listen(1)
c,a = s.accept()

Overwriting server0.py


- `s` here is a server socket. It binds to '', or any address on this machine, and port 25000. 
- Low number ports are reserved by system services, only the root user can create them.
- listen tells the socker to queue in only `1` request here before refusing outside connections. 5-10 is plenty, if your code is well written
- `accept` creates a client socket `c` with address `a`

In [3]:
%%file client0.py
from socket import *
c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))

Overwriting client0.py


From the client:

- when you are `connect`ed the socket `c` can be used to send in a request or to recieve some data. 
- This socket will read the response and then be destroyed. Client sockets are one-shot
- the client's client socket and server's client socket are the same type of thing, so you need to decide "who calls whom"

Communication happens using some variant of `send` and `recv`, or `read` and `write`. It is your responsibility to call and call again until the buffers are full. We do that below, and note that we are reading into already allocated buffers (many a time we will allocate on the fly, as we shall see).

This is a great example of memoryviews in Python as well.

### Sharing data with sockets

In [3]:
%%file client0.py
import numpy as np
from socket import *


def recv_into(arr, source):
    view = memoryview(arr).cast('B') 
    while len(view):
        nrecv = source.recv_into(view)
        print("recieved", nrecv)
        view = view[nrecv:]
            
c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))
a = np.zeros(shape=50000000, dtype=float)
print(a[0:10])
recv_into(a, c)
print(a[0:10])

Overwriting client0.py


In [4]:
%%file server0.py
from socket import *
import numpy as np

def send_from(arr, dest):
    view = memoryview(arr).cast('B') 
    while len(view):
        nsent = dest.send(view)
        view = view[nsent:]
            

s = socket(AF_INET, SOCK_STREAM)
s.bind(('', 25000))
s.listen(1)
c,addr = s.accept()
a = np.arange(0.0, 50000000.0)
send_from(a, c)

Overwriting server0.py


In this case we have chosen to have the server "write-to" the client. We could have had the client request the server. We'll do that next

Notice that so far we have not tried to keep the server socket persistent. We'll do this next as well. But first let us write the cacnonical echo server. We can test this with `telnet`.

### The canonical echo server

In [5]:
%%file echo_server.py
from socket import socket, AF_INET, SOCK_STREAM

def echo_handler(address, client_sock):
    print('Got connection from {}'.format(address)) 
    while True:
        msg = client_sock.recv(2)
        print(len(msg))
        if not msg:
            print("broke")
            break
        client_sock.sendall(msg)
    client_sock.close()

def echo_server(address, backlog=5): 
    sock = socket(AF_INET, SOCK_STREAM) 
    sock.bind(address) 
    sock.listen(backlog)
    while True:
        client_sock, client_addr = sock.accept() 
        echo_handler(client_addr, client_sock)
        
if __name__ == '__main__': 
    echo_server(('', 20002))

Writing echo_server.py


### Making the server persistent

In [6]:
%%file server1.py
from socket import *
import numpy as np

a = np.arange(0.0, 50000000.0)

def send_from(arr, dest):
    view = memoryview(arr).cast('B') 
    while len(view):
        nsent = dest.send(view)
        view = view[nsent:]
            
def handle(csock):
    msg = b""
    while True:
        recvd = csock.recv(2)#chosen to be large enough
        print("recieved", recvd)
        if not recvd: # handle close
            break
        msg += recvd
        if len(msg)==10:
            break
    offset, numele=msg.decode().split(':')
    offset=int(offset)
    numele=int(numele)
    send_from(a[offset:offset+numele], csock)
    csock.close()
        
def array_server(address_tuple, backlog=1):
    s = socket(AF_INET, SOCK_STREAM)
    s.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    s.bind(address_tuple)
    s.listen(backlog)
    
    while True:
        csock,caddr = s.accept()
        print("got connection from {}".format(caddr))
        handle(csock)
    s.close()

array_server(('', 25000))

Writing server1.py


In [7]:
%%file client1.py
import numpy as np
from socket import *
offset=5000
num_wanted=10000

def recv_into(arr, source):
    view = memoryview(arr).cast('B') 
    while len(view):
        nrecv = source.recv_into(view)
        print("recieved", nrecv)
        view = view[nrecv:]
            

c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))
mybytes=str.encode("{}:{}".format(offset, num_wanted))
print(len(mybytes))
c.send(mybytes)
a = np.zeros(shape=num_wanted, dtype=float)
print(a[0:10])
recv_into(a, c)
print(a[0:10])

Writing client1.py


## Using the socket server module

In [8]:
%%file echo_server2.py
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler) 
    serv.serve_forever()

Writing echo_server2.py


In [9]:
%%file echo_client.py
import sys
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 20000))
s.send(sys.argv[1].encode())
print(s.recv(8192))

Writing echo_client.py


### Threaded implementations

In [59]:
%%file echo_server3.py
from socketserver import BaseRequestHandler, ThreadingTCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = ThreadingTCPServer(('', 20000), EchoHandler) 
    serv.serve_forever()

Writing echo_server3.py


In [62]:
%%file echo_server4.py
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

if __name__ == '__main__':
    from threading import Thread
    NWORKERS = 16
    TCPServer.allow_reuse_address = True
    serv = TCPServer(('', 20000), EchoHandler) 
    
    for n in range(NWORKERS):
            t = Thread(target=serv.serve_forever)
            t.daemon = True
            t.start()
    serv.serve_forever()

Overwriting echo_server4.py


Here is the usual echo server written with a thread pool from the `concurrent.futures` package. We saw some other ways of writing this server above using the `socketserver` module

In [10]:
%%file threads0.py
from socket import AF_INET, SOCK_STREAM, socket 
from concurrent.futures import ThreadPoolExecutor

def echo_client(sock, client_addr):
    print('Got connection from', client_addr) 
    while True:
        msg = sock.recv(65536) 
        if not msg:
            break
        sock.sendall(msg) 
    print('Client closed connection') 
    sock.close()
    
def echo_server(addr):
    pool = ThreadPoolExecutor(12) 
    sock = socket(AF_INET, SOCK_STREAM) 
    sock.bind(addr)
    sock.listen(5)
    while True:
        client_sock, client_addr = sock.accept()
        pool.submit(echo_client, client_sock, client_addr)
        
echo_server(('',15000))

Writing threads0.py


And here is something more along the lines of what we did last time in setting up a thread pool, but you can see how the pool is set up and fed.

In [11]:
%%file threads1.py
from socket import socket, AF_INET, SOCK_STREAM 
from threading import Thread
from queue import Queue

def echo_client(q):
    sock, client_addr = q.get()
    print('Got connection from', client_addr) 
    while True:
        msg = sock.recv(65536) 
        if not msg:
            break
        sock.sendall(msg) 
    print('Client closed connection')
    sock.close()

def echo_server(addr, nworkers): 
    # Launch the client workers 
    q = Queue()
    for n in range(nworkers):
        t = Thread(target=echo_client, args=(q,))
        t.daemon = True
        t.start()
    # Run the server
    sock = socket(AF_INET, SOCK_STREAM) sock.bind(addr)
    sock.listen(5)
    while True:
        client_sock, client_addr = sock.accept()
        q.put((client_sock, client_addr))
    
echo_server(('',15000), 128)

Writing threads1.py


So lets use this model to write a simple, in-memory database server.

In [13]:
%%file dbserver.py
from socket import AF_INET, SOCK_STREAM, socket, SOL_SOCKET, SO_REUSEADDR
from concurrent.futures import ThreadPoolExecutor
import threading
class LockableDict: 
    def __init__(self):
        self._d={}
        self._dlock={}
        
    def __getitem__(self, attr):
        return self._d[attr]
    
    def __setitem__(self, attr, val):
        if attr not in self._d:
            self._dlock[attr]=threading.Lock()
        print("LOCKING FOR", attr, val)
        with self._dlock[attr]:
            self._d[attr] = val
        print("UNLOCKED FOR", attr, val)
            
def db_client(sock, client_addr, ldict):
    print('Got connection from', client_addr) 
    while True:
        msg = sock.recv(65536)
        print("msg", msg)
        if not msg:
            break
        key, value = msg.decode().split('=')
        print("k,v", key, value)
        ldict[key] = value
        sock.sendall(value.encode())
    print('Client closed connection') 
    sock.close()
    
def db_server(addr):
    print("creating lockable dict and pool")
    ldict=LockableDict()
    pool = ThreadPoolExecutor(50) 
    sock = socket(AF_INET, SOCK_STREAM)
    sock.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    sock.bind(addr)
    sock.listen(15)
    while True:
        print('connection')
        client_sock, client_addr = sock.accept()
        pool.submit(db_client, client_sock, client_addr, ldict)
        
db_server(('',15000))

Overwriting dbserver.py


Here's a client that uses a threadpool

In [14]:
%%file dbclient.py
import sys
from socket import socket, AF_INET, SOCK_STREAM
from concurrent.futures import ThreadPoolExecutor
def fetch(i):
    s = socket(AF_INET, SOCK_STREAM)
    s.connect(('localhost', 15000))
    print("sending, i",i)
    s.send("a={}".format(i).encode())
    print("sent")
    return s.recv(65536)
pool = ThreadPoolExecutor(20)
thrs=[]
for i in range(40):
    t = pool.submit(fetch, i)
    thrs.append(t)
for i in range(40):
    print('i', i, thrs[i].result())

Writing dbclient.py


And another that uses a process pool

In [15]:
%%file dbclient2.py
import sys
from socket import socket, AF_INET, SOCK_STREAM
from concurrent.futures import ProcessPoolExecutor
def fetch(i):
    s = socket(AF_INET, SOCK_STREAM)
    s.connect(('localhost', 15000))
    print("sending, i",i)
    s.send("a={}".format(i).encode())
    print("sent")
    return s.recv(65536)
pool = ProcessPoolExecutor(20)
thrs=[]
for i in range(40):
    t = pool.submit(fetch, i)
    thrs.append(t)
for i in range(40):
    print('i', i, thrs[i].result())

Writing dbclient2.py


And a third that uses the process pool using the built in map method, just for illustration.

In [16]:
%%file dbclient3.py
import sys
from socket import socket, AF_INET, SOCK_STREAM
from concurrent.futures import ProcessPoolExecutor
def fetch(i):
    s = socket(AF_INET, SOCK_STREAM)
    s.connect(('localhost', 15000))
    print("sending, i",i)
    s.send("a={}".format(i).encode())
    print("sent")
    return s.recv(65536)

with ProcessPoolExecutor(20) as pool: 
    for result in pool.map(fetch, range(40)):
        print(result)

Writing dbclient3.py


Instead of blocking, one can use callbacks. For example (from the cookbook):

```python
def when_done(r): 
    print('Got:', r.result())
with ProcessPoolExecutor() as pool: 
        future_result = pool.submit(work, arg)
        future_result.add_done_callback(when_done)
        
```

We do not show how we can create a multiprocessing based db server. But its doable. Three things to keep in mind

(a) you can create the database by doing a multiprocessing.lock (b) you must however do this in shared memory (c) you cannot easily pass the socket in the client function like we did above. This is as function arguments are passed from one process to the other by pickling, and sockets are not pickle-able. This leaves you with the choice of sending the data only to a child process, or using multiprocessing.reduction to somehow pickle the socket, or preforking (what apache does) in which the accept is run in the client (see http://foobarnbaz.com/2011/08/30/developing-scalable-services-with-python/) for an example.


#### Daemonization
You want your process to run afteryou kill your shell. See http://chimera.labs.oreilly.com/books/1230000000393/ch12.html#_problem_210 .

### Writing a web page fetcher

 
Adapted from http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html

It talks about callback and co-routines, how non-blocking socket based programming works, and some of the new functionality in python 3.4+. Very much worth reading.

#### Blocking fetch

In [15]:
import socket
def fetch(host, url):
    sock = socket.socket()
    sock.connect((host, 80))
    request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(url, host)
    sock.send(request.encode('ascii'))
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        chunk = sock.recv(4096)
    return response

In [17]:
str(fetch("www.example.com","/"))

'b\'HTTP/1.0 200 OK\\r\\nCache-Control: max-age=604800\\r\\nContent-Type: text/html\\r\\nDate: Wed, 09 Nov 2016 07:51:27 GMT\\r\\nEtag: "359670651+gzip+ident"\\r\\nExpires: Wed, 16 Nov 2016 07:51:27 GMT\\r\\nLast-Modified: Fri, 09 Aug 2013 23:54:35 GMT\\r\\nServer: ECS (bos/F56E)\\r\\nVary: Accept-Encoding\\r\\nX-Cache: HIT\\r\\nx-ec-custom-error: 1\\r\\nContent-Length: 1270\\r\\nConnection: close\\r\\n\\r\\n<!doctype html>\\n<html>\\n<head>\\n    <title>Example Domain</title>\\n\\n    <meta charset="utf-8" />\\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\\n    <style type="text/css">\\n    body {\\n        background-color: #f0f0f2;\\n        margin: 0;\\n        padding: 0;\\n        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\\n        \\n    }\\n    div {\\n        width: 600px;\\n        margin: 5em auto;\\n        padding: 50px;\\n        backgr

#### Exercise

You can run the fetches in threads or processes. For the threaded implementation, a queue can be used to collect the results. For multi-process, Redis is a good choice, or you'll need something shared-memory.